Wikipedia Web crawler

This section performs a search on Wikipedia for a given query and fetches the article content.

In [1]:
import requests
from bs4 import BeautifulSoup
import json


# Search from wikipedia
def search_wikipedia(query):
    params = {
        'action': 'query',
        'list': 'search',
        'srsearch': query,
        'format': 'json',
        'srlimit': 1
    }
    response = requests.get("https://en.wikipedia.org/w/api.php", params=params)
    return response.json().get('query', {}).get('search', []) if response.status_code == 200 else []

# Fetch article's content
def fetch_article_content(title):
    url = f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')
    content = soup.find('div', class_='mw-parser-output')
    return ' '.join([p.text for p in content.find_all('p') if p.text.strip()]) if content else None

# Main function to fetch articles and save them to a json
def fetch_and_save_articles(queries, filename="wikipedia_articles.json"):
    articles = []
    for query in queries:
        search_results = search_wikipedia(query)
        for result in search_results:
            content = fetch_article_content(result['title'])
            print(result['title'])
            if content:
                articles.append({'title': result['title'], 'content': content})
    if articles:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(articles, f, ensure_ascii=False, indent=4)
        print(f"Saved {len(articles)} articles to {filename}.")
    else:
        print("No articles found.")


def main():
    queries = "Deep learning,Document classification,Information retrieval,Natural language processing,Learning to rank,Information retrieval,Recommender system,Large language model,Prompt engineering,Machine learning,Recurrent neural network,Ranking Algorithms,Climate Change,History of Art,Health and Nutrition,The History of the Olympic Games,Cave exploration".split(",")
    fetch_and_save_articles(queries)

if __name__ == "__main__":
    main()

Deep learning
Document classification
Information retrieval
Natural language processing
Learning to rank
Information retrieval
Recommender system
Large language model
Prompt engineering
Machine learning
Recurrent neural network
Search algorithm
Climate change
History of art
Nutrition and Health
Olympic Games
Caving
Saved 15 articles to wikipedia_articles.json.


Process articles

This section preprocesses the articles by removing stop-words and saves them in a JSON file.

In [2]:
import json
import re

STOP_WORDS = {
    "a", "an", "and", "the", "is", "in", "it", "of", "to", "with", "on", "for", "this", "that", "at", "by", "from",
    "as", "are", "be", "or", "but", "not", "was", "which", "so", "if", "can", "will", "would", "has", "have", "had"
}

def preprocess_text(text):
    """Preprocesses text by removing special characters, converting to lowercase, and remove stop-words."""
    return ' '.join(
        token for token in re.sub(r'[^a-zA-Z\s]', '', text).lower().split() 
        if token not in STOP_WORDS
    )

def read_json_file(filename):
    """Reads JSON file and returns its content or None on failure."""
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error: {e}")
        return None

def process_articles(input_file, output_file):
    """Processes articles from input JSON file and saves preprocessed data to output JSON file."""
    if articles := read_json_file(input_file):
        processed_articles = [
            {'title': article.get('title', 'No Title'), 
             'content': preprocess_text(article.get('content', ''))}
            for article in articles
        ]
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(processed_articles, f, ensure_ascii=False, indent=4)
        print(f"Processed articles saved in '{output_file}'.")
    else:
        print("No articles to process.")


def main():
    process_articles("wikipedia_articles.json", "processed_wikipedia_articles.json")

if __name__ == "__main__":
    main()

Processed articles saved in 'processed_wikipedia_articles.json'.


Inverted index

This section creates an inverted index from the processed articles and saves it in a JSON file.

In [3]:
import json
from collections import defaultdict

# Loads preprocessed articles from a JSON file
def load_processed_articles(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            articles = json.load(file)
            print(f"Loaded {len(articles)} articles from '{filename}'.")
            return articles
    except Exception as e:
        print(f"Error loading file: {e}")
        return []

# Creates an inverted index from a list of articles
def create_inverted_index(articles):
    inverted_index = defaultdict(list)
    for article_id, article in enumerate(articles):
        for word in article.get('content', '').split():
            if article_id not in inverted_index[word]:
                inverted_index[word].append(article_id)
    return inverted_index

# Saves the inverted index to a JSON file
def save_inverted_index(inverted_index, filename="inverted_index.json"):
    try:
        with open(filename, 'w', encoding='utf-8') as file:
            json.dump(inverted_index, file, ensure_ascii=False, indent=4)
        print(f"Inverted index saved in '{filename}'.")
    except Exception as e:
        print(f"Error saving file: {e}")


def main():
    input_filename = "processed_wikipedia_articles.json"
    output_filename = "inverted_index.json"
    articles = load_processed_articles(input_filename)
    save_inverted_index(create_inverted_index(articles), output_filename)

if __name__ == "__main__":
    main()

Loaded 15 articles from 'processed_wikipedia_articles.json'.
Inverted index saved in 'inverted_index.json'.


Search engine

This section implements different search algorithms: Boolean Search, TF-IDF Search, and BM25 Search.

In [4]:
import json
import re
import numpy as np
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi

def load_json(filename):
    """Load a JSON file."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except Exception as e:
        print(f"Error loading '{filename}': {e}")
        return None

def preprocess_query(query):
    """Clean and tokenize the query."""
    return re.sub(r'[^a-zA-Z\s]', '', query).lower().split()

def boolean_search(query_tokens, inverted_index):
    """Perform boolean search on the inverted index."""
    results, operation = set(), "OR"
    found_results = False  # Flag to check if we found any results

    for token in query_tokens:
        if token.upper() in ["AND", "OR", "NOT"]:
            operation = token.upper()
        else:
            postings = set(inverted_index.get(token, []))
            if not postings:  # If no postings found for this token, skip it
                continue

            if operation == "OR":
                results |= postings
                found_results = True
            elif operation == "AND":
                if not results:  # If no previous results, initialize with the first set
                    results = postings
                else:
                    results &= postings
            elif operation == "NOT":
                results -= postings

    # If no results are found, return an empty set
    return results if found_results else set()


def tfidf_search(query_tokens, articles, threshold=0.1):
    """Perform TF-IDF search on the articles."""
    corpus = [article['content'] for article in articles]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    query_vector = vectorizer.transform([' '.join(query_tokens)])
    scores = np.dot(query_vector, tfidf_matrix.T).toarray().flatten()

    # Only return results with scores above the threshold
    return [i for i, score in enumerate(scores) if score > threshold]

def bm25_search(query_tokens, articles, threshold=0.1):
    """Perform BM25 search on the articles."""
    corpus = [article['content'].split() for article in articles]
    bm25 = BM25Okapi(corpus)
    scores = bm25.get_scores(query_tokens)

    # Only return results with scores above the threshold
    return [i for i, score in enumerate(scores) if score > threshold]


def search_query(query, inverted_index, articles, algorithm):
    """Search the query using the selected algorithm."""
    query_tokens = preprocess_query(query)
    if algorithm == "boolean":
        matched_ids = boolean_search(query_tokens, inverted_index)
        return [articles[doc_id] for doc_id in matched_ids] if matched_ids else []
    elif algorithm == "tfidf":
        matched_ids = tfidf_search(query_tokens, articles)
        return [articles[doc_id] for doc_id in matched_ids] if matched_ids else []
    elif algorithm == "bm25":
        matched_ids = bm25_search(query_tokens, articles)
        return [articles[doc_id] for doc_id in matched_ids] if matched_ids else []
    return []

def search_interface():
    """Search interface to interact with the user."""
    inverted_index = load_json("inverted_index.json")
    articles = load_json("processed_wikipedia_articles.json")
    if not inverted_index or not articles:
        print("Error loading data. Exiting...")
        return

    algorithms = {"1": "boolean", "2": "tfidf", "3": "bm25"}
    while True:
        choice = input("\nChoose algorithm (1: Boolean, 2: TF-IDF, 3: BM25, 'exit' to quit): ")
        if choice.lower() == "exit":
            print("Exiting search engine. Goodbye!")
            break
        algorithm = algorithms.get(choice)
        if not algorithm:
            print("Invalid choice. Try again.")
            continue

        query = input("Enter your query: ")
        results = search_query(query, inverted_index, articles, algorithm)
        if results:
            print(f"\nFound {len(results)} result(s):")
            for result in results:
                print(f"- {result['title']}")
        else:
            print("No results found.")


def main():
    search_interface()

if __name__ == "__main__":
    main()


Found 6 result(s):
- Deep learning
- Large language model
- Prompt engineering
- Machine learning
- Recurrent neural network
- History of art

Found 1 result(s):
- Caving

Found 7 result(s):
- Deep learning
- Natural language processing
- Recommender system
- Large language model
- Prompt engineering
- Machine learning
- Search algorithm
Exiting search engine. Goodbye!


Evaluation

This section evaluates the effectiveness of different search algorithms: Boolean Search, TF-IDF Search, and BM25 Search. Each algorithm is evaluated using precision, recall, F1-Score, and mean average precision (MAP) based on test queries and relevant documents.

In [5]:
import json
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings("ignore")

# Load inverted index
def load_inverted_index(filename="inverted_index.json"):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except Exception as e:
        print(f"Error loading inverted index: {e}")
        return None

# Load articles
def load_articles(filename="processed_wikipedia_articles.json"):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            articles = json.load(file)
            return articles
    except Exception as e:
        print(f"Error loading articles: {e}")
        return None


# Query processing
def preprocess_query(query):
    query = re.sub(r'[^a-zA-Z\s]', '', query)  
    query = query.lower()  
    tokens = query.split() 
    return tokens

# Boolean search
def boolean_search(query_tokens, inverted_index):
    results = set()
    operation = "OR"  # if there are not logical operators OR is used

    for token in query_tokens:
        if token.upper() in ["AND", "OR", "NOT"]:
            operation = token.upper()
        else:
            postings = set(inverted_index.get(token, []))
            
            if operation == "OR":
                results |= postings
            elif operation == "AND":
                results &= postings
            elif operation == "NOT":
                results -= postings

    return results

#  TF-IDF search
def tfidf_search(query_tokens, articles):

    # Get all document content
    documents = [article['content'] for article in articles]
    
    # Create vectorizer TF-IDF
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(documents)
    
    # Create query vector
    query = ' '.join(query_tokens)
    query_vector = vectorizer.transform([query])

    # cosine similarity
    cosine_similarities = np.array(X.dot(query_vector.T).toarray()).flatten()
    
    return cosine_similarities

def bm25_search(query_tokens, articles):
    # Tokenize the articles' text
    tokenized_articles = [word_tokenize(article['content'].lower()) for article in articles]
    
    # Initialize the BM25 model
    bm25 = BM25Okapi(tokenized_articles)
    
    # Score the query against all documents
    scores = bm25.get_scores(query_tokens)
    
    return scores

# Evaluation
def evaluate_search_algorithm(query, relevant_docs, inverted_index, articles, algorithm):

    query_tokens = preprocess_query(query)
    
    if algorithm == "boolean":
        retrieved_docs = boolean_search(query_tokens, inverted_index)
    elif algorithm == "tfidf":
        cosine_similarities = tfidf_search(query_tokens, articles)
        ranked_docs = sorted(enumerate(cosine_similarities), key=lambda x: x[1], reverse=True)
        retrieved_docs = [doc_id for doc_id, _ in ranked_docs]
    elif algorithm == "bm25":
        scores = bm25_search(query_tokens, articles)
        ranked_docs = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
        retrieved_docs = [doc_id for doc_id, _ in ranked_docs]
    else:
        return None

    # Υπολογισμός ακρίβειας, ανάκλησης και F1-score
    # if there are not related articles 0 the variables
    y_true = [1 if doc_id in relevant_docs else 0 for doc_id in retrieved_docs]
    y_pred = [1 if doc_id in relevant_docs else 0 for doc_id in retrieved_docs]

    if sum(y_true) == 0 and sum(y_pred) == 0:
        precision = recall = f1 = 0.0
    else:
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)

    # Calculate (MAP)
    if len(retrieved_docs) == 0:
        ap = 0.0
    else:
        ap = average_precision_score(y_true, y_pred)

    return precision, recall, f1, ap

# Create test queries, uncomment one of them only to test cases
def generate_test_queries():
    # test_queries = [
    #     {"query": "machine learning", "relevant_docs": [0, 9, 7]},
    #     {"query": "information retrieval", "relevant_docs": [2, 5, 4]},
    #     {"query": "natural language processing", "relevant_docs": [3, 7, 8]},
    #     {"query": "deep learning", "relevant_docs": [0, 10, 7]},
    #     {"query": "search algorithm", "relevant_docs": [11, 2]},
    #     {"query": "recommender system", "relevant_docs": [6, 4]},
    #     {"query": "prompt engineering", "relevant_docs": [8, 7]},
    #     {"query": "learning to rank", "relevant_docs": [4, 2]},
    # ]
    test_queries = [
        {"query": "What is machine learning?", "relevant_docs": [0, 9]},
        {"query": "Techniques for recommending items", "relevant_docs": [6, 8]},
        {"query": "How to process natural language?", "relevant_docs": [3, 7]},
        {"query": "Algorithms for ranking results", "relevant_docs": [4, 5]},
        {"query": "What are large language models used for?", "relevant_docs": [7, 8]},
        {"query": "Deep learning in AI systems", "relevant_docs": [0, 1]},
        {"query": "How to design search algorithms?", "relevant_docs": [11, 4]},
        {"query": "Applications of recurrent neural networks", "relevant_docs": [9, 10]},
        {"query": "Methods for improving information retrieval", "relevant_docs": [5, 3]},
        {"query": "How to engineer effective prompts?", "relevant_docs": [8, 7]}
    ]
    # test_queries = [
    #     {"query": "applications of deep learning in natural language processing", "relevant_docs": [0, 3, 9]},
    #     {"query": "methods for document classification and information retrieval", "relevant_docs": [1, 2, 4]},
    #     {"query": "techniques in recommender systems and large language models", "relevant_docs": [6, 7, 8]},
    #     {"query": "overview of machine learning and recurrent neural networks", "relevant_docs": [9, 10]},
    #     {"query": "ranking algorithms in search engines", "relevant_docs": [11, 2, 4]},
    #     {"query": "climate change impact on global ecosystems", "relevant_docs": [12]},
    #     {"query": "renaissance art history and its influence", "relevant_docs": [13]},
    #     {"query": "importance of balanced diet in health and nutrition", "relevant_docs": [14]},
    #     {"query": "history and significance of the Olympic Games", "relevant_docs": [15]},
    #     {"query": "exploration techniques in cave environments", "relevant_docs": [16]}
    # ]
    # test_queries = [
    #     {"query": "machine AND learning", "relevant_docs": [0, 10]},
    #     {"query": "deep learning OR artificial intelligence", "relevant_docs": [2, 8, 10]},
    #     {"query": "natural language processing NOT deep learning", "relevant_docs": [3, 4, 5]},
    #     {"query": "machine learning AND (deep learning OR neural networks) NOT artificial intelligence", "relevant_docs": [0, 2, 8]}
    # ]
    # test_queries = [
    #     {"query": "deep learning AND artificial intelligence AND health and nutrition", "relevant_docs": [0, 14, 16]},
    #     {"query": "machine learning AND climate change AND renaissance art", "relevant_docs": [9, 12, 13]},
    #     {"query": "information retrieval AND history of art AND cave exploration", "relevant_docs": [2, 13, 16]},
    #     {"query": "machine learning AND recommender systems AND neural networks", "relevant_docs": [9, 6, 10]},
    #     {"query": "deep learning AND large language models AND health and nutrition", "relevant_docs": [0, 7, 14]},
    #     {"query": "search algorithms AND climate change AND prompt engineering", "relevant_docs": [11, 12, 8]},
    #     {"query": "natural language processing AND document classification AND deep learning AND cave exploration", "relevant_docs": [3, 1, 0, 16]},
    #     {"query": "ranking algorithms AND recommender systems AND health and nutrition", "relevant_docs": [11, 6, 14]},
    #     {"query": "history of art AND large language models AND artificial intelligence", "relevant_docs": [13, 7, 0]},
    #     {"query": "machine learning AND deep learning NOT artificial intelligence", "relevant_docs": [9, 0]},
    #     {"query": "natural language processing AND recommender systems NOT machine learning", "relevant_docs": [3, 6]},
    #     {"query": "climate change AND health and nutrition NOT deep learning", "relevant_docs": [12, 14]},
    #     {"query": "history of art AND health and nutrition AND artificial intelligence", "relevant_docs": [13, 14, 0]},
    #     {"query": "climate change AND cave exploration AND prompt engineering", "relevant_docs": [12, 16, 8]},
    #     {"query": "renaissance art AND machine learning AND recommender systems", "relevant_docs": [13, 9, 6]},
    #     {"query": "recommender systems", "relevant_docs": [6]},
    #     {"query": "ranking algorithms", "relevant_docs": [11]},
    #     {"query": "health", "relevant_docs": [14]},
    #     {"query": "deep learning AND large language models AND healthcare policies", "relevant_docs": [0, 7, 14]},
    #     {"query": "artificial intelligence AND climate change AND medieval history", "relevant_docs": [0, 12, 13]},
    #     {"query": "natural language processing AND cave exploration AND health", "relevant_docs": [3, 16, 14]}
    # ]

    return test_queries

# Evaluate for all algorithms
def evaluate_system():
    inverted_index = load_inverted_index()
    articles = load_articles()

    if not inverted_index or not articles:
        print("Error loading data. Exiting...")
        return

    test_queries = generate_test_queries()

    algorithms = ["boolean", "tfidf", "bm25"]
    for algorithm in algorithms:
        print(f"\nEvaluating using {algorithm} search...")
        
        precision_total = recall_total = f1_total = ap_total = 0
        num_queries = len(test_queries)
        
        for query_data in test_queries:
            query = query_data["query"]
            relevant_docs = query_data["relevant_docs"]
            
            precision, recall, f1, ap = evaluate_search_algorithm(query, relevant_docs, inverted_index, articles, algorithm)
            
            precision_total += precision
            recall_total += recall
            f1_total += f1
            ap_total += ap

            print(f"Query: '{query}' - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}, MAP: {ap:.4f}")
        
        print(f"\nAverage Precision: {precision_total / num_queries:.4f}")
        print(f"Average Recall: {recall_total / num_queries:.4f}")
        print(f"Average F1-Score: {f1_total / num_queries:.4f}")
        print(f"Average MAP: {ap_total / num_queries:.4f}")


def main():
    evaluate_system()

if __name__ == "__main__":
    main()


Evaluating using boolean search...
Query: 'What is machine learning?' - Precision: 1.0000, Recall: 1.0000, F1-Score: 1.0000, MAP: 1.0000
Query: 'Techniques for recommending items' - Precision: 1.0000, Recall: 1.0000, F1-Score: 1.0000, MAP: 1.0000
Query: 'How to process natural language?' - Precision: 1.0000, Recall: 1.0000, F1-Score: 1.0000, MAP: 1.0000
Query: 'Algorithms for ranking results' - Precision: 1.0000, Recall: 1.0000, F1-Score: 1.0000, MAP: 1.0000
Query: 'What are large language models used for?' - Precision: 1.0000, Recall: 1.0000, F1-Score: 1.0000, MAP: 1.0000
Query: 'Deep learning in AI systems' - Precision: 1.0000, Recall: 1.0000, F1-Score: 1.0000, MAP: 1.0000
Query: 'How to design search algorithms?' - Precision: 1.0000, Recall: 1.0000, F1-Score: 1.0000, MAP: 1.0000
Query: 'Applications of recurrent neural networks' - Precision: 1.0000, Recall: 1.0000, F1-Score: 1.0000, MAP: 1.0000
Query: 'Methods for improving information retrieval' - Precision: 1.0000, Recall: 1.0000